In [4]:
import json
import folium
from branca.colormap import linear
from pathlib import Path
from folium.plugins import HeatMap


In [2]:
m = folium.Map(location=[33.7490, -84.3880], zoom_start=12)

In [3]:
m

In [2]:
desirable_path = Path("../../data/map/desirable_undesirable_activities/desirable_undesirable_scores.geojson")
stable_path    = Path("../../data/map/stable_communities/stable_communities_scores_geo.geojson")

In [3]:
with desirable_path.open() as f:
    desir_geo = json.load(f)

with stable_path.open() as f:
    stab_geo = json.load(f)

In [4]:
def _get_vals(features, prop_name):
    """Return list of property values (ignores null)."""
    return [f["properties"][prop_name]
            for f in features
            if f["properties"].get(prop_name) is not None]

In [5]:
# min/max for each layer (needed for color scales)
desir_vals = _get_vals(desir_geo["features"], "final_score")
stab_vals  = _get_vals(stab_geo["features"],  "best_score")

In [6]:
desir_cmap = linear.YlGnBu_09.scale(min(desir_vals), max(desir_vals))
stab_cmap  = linear.Purples_09.scale(min(stab_vals),  max(stab_vals))

In [7]:
# Start folium map 
# center on GA roughly; tweak if you want a tighter bbox
m = folium.Map(location=[32.5, -83.5], zoom_start=7, tiles="CartoDB positron")

In [8]:
# Add Desirable/Undesirable grid 
folium.GeoJson(
    desir_geo,
    name="Desirable / Undesirable",
    style_function=lambda feat: {
        "fillColor"  : desir_cmap(feat["properties"]["final_score"]),
        "color"      : "#555555",
        "weight"     : 0.3,
        "fillOpacity": 0.8,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=["final_score", "desirable_score", "total_deductions"],
        aliases=["Final", "Desirable", "Deductions"],
        localize=True,
    ),
).add_to(m)
desir_cmap.caption = "Final score (Desirable / Undesirable)"
desir_cmap.add_to(m)


In [9]:
# Add Stable Communities grid 
folium.GeoJson(
    stab_geo,
    name="Stable Communities",
    style_function=lambda feat: {
        "fillColor"  : stab_cmap(feat["properties"]["best_score"]),
        "color"      : "#555555",
        "weight"     : 0.3,
        "fillOpacity": 0.8,
    },
    tooltip=folium.features.GeoJsonTooltip(
        fields=["best_score", "use_only_actual_tract", "use_nearby_tract"],
        aliases=["Best", "Actual tract", "Nearby tract"],
        localize=True,
    ),
).add_to(m)
stab_cmap.caption = "Best score (Stable Communities)"
stab_cmap.add_to(m)

In [ ]:
# Laayer toggle & output 
folium.LayerControl(collapsed=False).add_to(m)
m.save("lihtc_score_layers.html")
m

In [5]:
# -------------------------------------------------------------------
# 0.  one‑time install (if needed):
#     pip install folium branca
# -------------------------------------------------------------------
import json, pathlib, folium
from branca.colormap import linear
from IPython.display import display
import json
import folium
from branca.colormap import linear
from pathlib import Path

# -------------------------------------------------------------------
# 1.  Load your GeoJSON layers
# -------------------------------------------------------------------
d_path = Path("../../data/map/desirable_undesirable_activities/desirable_undesirable_scores.geojson")
s_path    = Path("../../data/map/stable_communities/stable_communities_scores_geo.geojson")

desir = json.loads(d_path.read_text())
stab  = json.loads(s_path.read_text())

# ------------- function that makes a Folium layer ------------------
def build_layer(geojson, val_field, palette):
    vals = [f["properties"][val_field] for f in geojson["features"]]
    cmap = palette.scale(min(vals), max(vals))
    return folium.GeoJson(
        geojson,
        name=val_field,
        style_function=lambda f: {
            "fillColor": cmap(f["properties"][val_field]),
            "color": "#555555", "weight": 0.3, "fillOpacity": 0.8,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=[val_field, "lat", "lon"],
            aliases=["Score", "Lat", "Lon"],
            localize=True),
    ), cmap

# -------------------------------------------------------------------
# 2.  SMALL‑sample map *just* for notebook preview -------------------
# -------------------------------------------------------------------
desir_sample = desir.copy()
desir_sample["features"] = desir_sample["features"][:1_000]   # ⇠ preview size

m_preview = folium.Map(
    location=[32.5, -83.5],
    zoom_start=7,
    tiles="OpenStreetMap",
    width="100%", height="600px"
)

layer, cmap = build_layer(desir_sample, "final_score", linear.YlGnBu_09)
layer.add_to(m_preview); cmap.add_to(m_preview)
folium.LayerControl().add_to(m_preview)

display(m_preview)           

# # -------------------------------------------------------------------
# # 3.  FULL map written to disk --------------------------------------
# # -------------------------------------------------------------------
# m_full = folium.Map(location=[32.5, -83.5], zoom_start=7, tiles="OpenStreetMap")

# layer_d, cmap_d = build_layer(desir, "final_score",   linear.YlGnBu_09)
# layer_s, cmap_s = build_layer(stab,  "best_score",    linear.Purples_09)

# layer_d.add_to(m_full); cmap_d.add_to(m_full)
# layer_s.add_to(m_full); cmap_s.add_to(m_full)

# heat_data_full = [
#     [f["properties"]["lat"], f["properties"]["lon"], f["properties"]["final_score"]]
#     for f in desir["features"]
#     if f["properties"]["final_score"] is not None
# ]

# HeatMap(heat_data_full, name="Heatmap (Final Score)", radius=10, blur=15).add_to(m_full)


# folium.LayerControl(collapsed=False).add_to(m_full)

# out_html = "lihtc_full.html"
# m_full.save(out_html)
# print(f"\n✅  Full map written to {out_html}.  "
#       "Double‑click it or run `python -m http.server` and open in your browser.")

# FULL map written to disk
m_full = folium.Map(location=[32.5, -83.5], zoom_start=7, tiles="OpenStreetMap")

layer_d, cmap_d = build_layer(desir, "final_score",   linear.YlGnBu_09)
layer_s, cmap_s = build_layer(stab,  "best_score",    linear.Purples_09)

layer_d.add_to(m_full); cmap_d.add_to(m_full)
layer_s.add_to(m_full); cmap_s.add_to(m_full)

# Add heatmap
heat_data_full = [
    [f["properties"]["lat"], f["properties"]["lon"], f["properties"]["final_score"]]
    for f in desir["features"]
    if f["properties"]["final_score"] is not None
]
HeatMap(heat_data_full, name="Heatmap (Final Score)", radius=10, blur=15).add_to(m_full)

folium.LayerControl(collapsed=False).add_to(m_full)

out_html = "lihtc_full.html"
m_full.save(out_html)
print(f"\n✅  Full map written to {out_html}.  "
      "Double‑click it or run `python -m http.server` and open in your browser.")



✅  Full map written to lihtc_full.html.  Double‑click it or run `python -m http.server` and open in your browser.


In [6]:
desir_sample = desir.copy()
desir_sample["features"] = desir_sample["features"][:1_000]   # ⇠ preview size

m_preview = folium.Map(
    location=[32.5, -83.5],
    zoom_start=7,
    tiles="OpenStreetMap",
    width="100%", height="600px"
)

layer, cmap = build_layer(desir_sample, "final_score", linear.YlGnBu_09)
layer.add_to(m_preview)
cmap.add_to(m_preview)

# Add HeatMap
heat_data_preview = [
    [f["properties"]["lat"], f["properties"]["lon"], f["properties"]["final_score"]]
    for f in desir_sample["features"]
    if f["properties"]["final_score"] is not None
]
HeatMap(
    heat_data_preview,
    name="Heatmap (Final Score)",
    radius=10,
    blur=15,
    min_opacity=0.3,
    max_zoom=14
).add_to(m_preview)

folium.LayerControl().add_to(m_preview)
display(m_preview)

In [7]:
m_preview = folium.Map(
    location=[32.5, -83.5],
    zoom_start=7,
    tiles="OpenStreetMap",
    width="100%", height="600px"
)

heat_data_preview = [
    [f["properties"]["lat"], f["properties"]["lon"], f["properties"]["final_score"]]
    for f in desir_sample["features"]
    if f["properties"]["final_score"] is not None
]

HeatMap(
    heat_data_preview,
    name="Heatmap (Final Score)",
    radius=10,
    blur=15,
    min_opacity=0.3,
    max_zoom=14
).add_to(m_preview)

folium.LayerControl().add_to(m_preview)
display(m_preview)

In [8]:

# Create base map
m_full = folium.Map(location=[32.5, -83.5], zoom_start=7, tiles="OpenStreetMap")

# Prepare heatmap data
heat_data_full = [
    [f["properties"]["lat"], f["properties"]["lon"], f["properties"]["final_score"]]
    for f in desir["features"]
    if f["properties"]["final_score"] is not None
]

# Add smoothed heatmap
HeatMap(
    heat_data_full,
    name="Heatmap (Final Score)",
    radius=14,
    blur=25,
    min_opacity=0.4,
    max_zoom=10
).add_to(m_full)

folium.LayerControl(collapsed=False).add_to(m_full)

In [9]:
m_full